In [1]:
import os
# change dir for custom imports
os.chdir('../')

In [2]:
os.getcwd()

'C:\\Users\\clari\\Desktop\\M2 - Thesis\\Research\\Dr Jacques Bou Abdo\\Recommender System\\5 - Ensemble Learning Model\\Accuracy Metrics\\Group Validation'

In [3]:
import pickle
import numpy as np
import pandas as pd
import math
from helpers.dataset_helpers import get_genres_as_columns, get_all_genres_list
from metric.helpers import (
    get_user_genre_list,
    get_user_max_likelihood,
    get_ideal_rankings,
    build_all_likelihood_dict
)

dataset_name = 'ml-5m-nf-1-2-3-4'
# original data from the dataset
ratings = pd.read_csv('datasets/' + dataset_name + '/clean/ratings.csv')
movies = pd.read_csv('datasets/' + dataset_name + '/movies.csv', encoding='latin-1')

# data from recommender0
testset = pd.read_csv('output/' + dataset_name + '/test.csv')
trainset = pd.read_csv('output/' + dataset_name + '/train.csv')
rankings = pd.read_csv('output/' + dataset_name + '/rankings.csv')

# merge item data to the ratings df
test_df = testset.merge(movies, how='inner', on='movieId').sort_values(by='userId')
#ratings_df = ratings.merge(movies, how='inner', on='movieId').sort_values(by='userId')
rankings_df = rankings.merge(movies, how='inner', on='movieId').sort_values(by='rank')
#.drop(['title','genres_y'], axis=1)
ratings['user_cat'] = None
ratings['rating_group'] = None
ratings['item_cat'] = None
ratings['user_group'] = None
ratings['genres_x'] = None
ratings['timestamp_y'] = None
ratings['title_x'] = None
ratings['title_y'] = None
ratings['timestamp_x'] = None
ratings['genres_y'] = None

ratings_df = ratings.drop(['timestamp_x',	'title_x',	'genres_x',	'timestamp_y',	'title_y',	'genres_y'	], axis=1)
# create a new column for the genres count
test_df['count_genres'] = test_df.apply(lambda x: len(x['genres'].split('|')), axis=1)


In [4]:
#ratings_df = ratings_df.rename(columns={'genres_x':'genres', 'title_x':'title'}).copy()
#ratings_df.iloc[:,7:]
ratings_df

,Unnamed: 0.1,Unnamed: 0,Column1,userId,movieId,rating,user_cat,rating_group,item_cat,nf1,...,3&4,1&2&3,2&3&4,1&2&4,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,timestamp,title_y.1,genres_y.1
0,0,21976,446620,8619,54426,3.5,None,None,None,0,...,0,0,0,0,0,1,0,1269300682,12:08 East of Bucharest (A fost sau n-a fost?)...,Comedy|Drama
1,30,21974,612574,8619,4638,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1113270109,Jurassic Park III (2001),Action|Adventure|Sci-Fi|Thriller
2,31,21959,701769,8619,8505,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1174346819,"Eel, The (Unagi) (1997)",Drama|Romance
3,32,21960,618242,8619,2130,3.5,None,None,None,0,...,0,0,0,0,0,1,0,1137974196,Atlantic City (1980),Crime|Drama|Romance
4,33,21961,120327,8619,75443,3.5,None,None,None,0,...,0,0,0,0,0,1,0,1269203261,Invisible Stripes (1939),Crime|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,3166,425464,311078,162516,44004,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1408820482,Failure to Launch (2006),Comedy|Romance
3213,3165,425463,823186,162516,5952,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1175594311,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
3214,3164,425462,477006,162516,160,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1074040573,Congo (1995),Action|Adventure|Mystery|Sci-Fi
3215,3171,425470,129393,162516,2520,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1075736674,Airport (1970),Drama


In [5]:
# get all unique genres and users
genres = get_all_genres_list(ratings_df)
users = list(set(ratings_df.userId.to_list()))
rankings_df.drop(['title', 'genres'],axis=1)
# merge rankings with the real ratings
#rankings_hits = rankings_df.merge(ratings_df, how='left', on=['userId', 'movieId', 'genres']).sort_values(by=['userId', 'rank'])
rankings_hits = rankings_df.merge(ratings_df, how='left', on=['userId', 'movieId']).sort_values(by=['userId', 'rank'])
#rankings_hits = rankings_df
#rankings_hits = rankings_hits.drop(['title', 'timestamp'], axis=1).rename(columns={'genres_x':'genres'})
#rankings_hits = rankings_hits.drop(['timestamp'], axis=1).rename(columns={'genres_x':'genres'})
rankings_hits

,userId,movieId,prediction_x,rank,title_x,genres_x,Unnamed: 0.1,Unnamed: 0,Column1,rating,...,3&4,1&2&3,2&3&4,1&2&4,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,timestamp,title_y.1,genres_y.1
0,8619,99402,3.920971,1,Alois Nebel (2011),Animation|Crime|War,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,8619,58998,3.904058,2,Forgetting Sarah Marshall (2008),Comedy|Romance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,8619,32298,3.858332,3,Guess Who (2005),Comedy|Romance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,8619,178153,3.803753,4,Death Trance (2005),Action|Sci-Fi|Thriller,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,8619,135202,3.800667,5,Prime Suspect: Scent of Darkness (1995),(no genres listed),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,162516,106458,3.535159,6,"Tiger from Tjampa, The (Harimau Tjampa) (1953)",Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,162516,178483,3.529783,7,The Marchers (2013),Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,162516,97768,3.526452,8,Free Men (Les hommes libres) (2011),Drama|War,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,162516,7004,3.520524,9,Kindergarten Cop (1990),Action|Comedy|Crime|Thriller,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ratings_df[ratings_df['rating']==0.0]

,Unnamed: 0.1,Unnamed: 0,Column1,userId,movieId,rating,user_cat,rating_group,item_cat,nf1,...,3&4,1&2&3,2&3&4,1&2&4,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,timestamp,title_y.1,genres_y.1


In [7]:
dataset_name

'ml-5m-nf-1-2-3-4'

In [8]:
users

[78849,
 32778,
 106507,
 55189,
 93855,
 31776,
 24610,
 109731,
 8619,
 34987,
 75309,
 126637,
 29104,
 84146,
 30643,
 122803,
 46264,
 103611,
 47166,
 37438,
 156480,
 112321,
 124482,
 90691,
 115907,
 144322,
 143049,
 97994,
 137293,
 80974,
 136142,
 162516,
 20055,
 61912,
 88539,
 32349,
 150655,
 162271,
 93424,
 110971,
 62199,
 72315,
 153086,
 17783]

In [9]:
# build the likelihood dict (dictionary will be saved in ./output/dataset)
# can be build it once per dataset (if we run group validation multiple times and vary some parameters)
from recommenders.utils.timer import Timer
with Timer() as test_time:
    build_all_likelihood_dict(users, genres, ratings_df, dataset_name)
print("Took {} seconds for likelihood_dict.".format(test_time.interval))

Took 3.378264600003604 seconds for likelihood_dict.


In [10]:
# load user likelihood data
with open('output/' + dataset_name + '/likelihood.pkl', 'rb') as pkl_handle:
	likelihood_dict = pickle.load(pkl_handle)

In [11]:
likelihood_dict

defaultdict(dict,
            {'Sci-Fi': {78849: 0.04054054054054054,
              32778: 0.05511811023622047,
              106507: 0.03894736842105263,
              55189: 0.05336426914153132,
              93855: 0.03784570596797671,
              31776: 0.04017857142857143,
              24610: 0.04007633587786259,
              109731: 0.029449423815621,
              8619: 0.027450980392156862,
              34987: 0.020066889632107024,
              75309: 0.055900621118012424,
              126637: 0.03495145631067961,
              29104: 0.06951340615690169,
              84146: 0.028985507246376812,
              30643: 0.0356718192627824,
              122803: 0.07317073170731707,
              46264: 0.06896551724137931,
              103611: 0.027806385169927908,
              47166: 0.057692307692307696,
              37438: 0.09280742459396751,
              156480: 0.07675194660734148,
              112321: 0.08633093525179857,
              124482: 0.049046321525885

In [12]:
# # small example on user likelihood - could be skipped
# # get user likelihood value example
#user_id = 49403 # Was set to 12 by Wissam - #1 - 541 - #2 - 49403
#target_genre = 'Drama'

# get_user_max_likelihood(user_id, target_genre, genres, ratings_df)
# # user offline dict for likelihood
#likelihood_dict[target_genre][user_id]

### Clustering - kmeans

In [13]:
clustered_df = pd.read_csv('output/' + dataset_name + '/clusters.csv')

# group clusters into another dataframe with different representation
grouped_clusters = clustered_df.groupby('cluster')['userId'].apply(list).reset_index(name='users_list')
grouped_clusters['users_per_cluster'] = grouped_clusters.apply(lambda x: list(set(x.users_list)), axis=1)
grouped_clusters = grouped_clusters[['cluster', 'users_per_cluster']]

## Model evaluation - alpha-beta-ndcg

### get ideal rankings

In [14]:
discarded_users = []
ideal_rankings = []
k = 5
# get ideal rankings for every unique user in the dataset
for user in users:
    # check if ratings of user in the test_df is less than k
    if len(test_df[test_df['userId'] == user]) <= k:
        discarded_users.append(user)
        continue

    user_ideal_ranks = get_ideal_rankings(user, likelihood_dict, test_df, k=k)
    df = pd.DataFrame(user_ideal_ranks, columns=["userId", "movieId", "prediction", "rank"])
    ideal_rankings.append(df)

user_ideal_ranks_df = pd.concat(ideal_rankings)

# merge item data to the ratings df and then left joing to get real ratings
user_ideal_ranks_df = user_ideal_ranks_df.merge(movies, how='inner', on='movieId').sort_values(by='userId')
user_ideal_ranks_df = user_ideal_ranks_df.merge(ratings_df, how='left', on=['userId', 'movieId', 'genres', 'title']).sort_values(by=['userId', 'rank'])
user_ideal_ranks_df = user_ideal_ranks_df.drop(['title', 'timestamp'], axis=1)

### alpha-beta-nDCG - Test data

In [15]:
rankings_hits = rankings_hits.rename(columns={'genres_x':'genres'}).copy()

In [16]:
ratings_df

,Unnamed: 0.1,Unnamed: 0,Column1,userId,movieId,rating,user_cat,rating_group,item_cat,nf1,...,3&4,1&2&3,2&3&4,1&2&4,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,timestamp,title_y.1,genres_y.1
0,0,21976,446620,8619,54426,3.5,None,None,None,0,...,0,0,0,0,0,1,0,1269300682,12:08 East of Bucharest (A fost sau n-a fost?)...,Comedy|Drama
1,30,21974,612574,8619,4638,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1113270109,Jurassic Park III (2001),Action|Adventure|Sci-Fi|Thriller
2,31,21959,701769,8619,8505,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1174346819,"Eel, The (Unagi) (1997)",Drama|Romance
3,32,21960,618242,8619,2130,3.5,None,None,None,0,...,0,0,0,0,0,1,0,1137974196,Atlantic City (1980),Crime|Drama|Romance
4,33,21961,120327,8619,75443,3.5,None,None,None,0,...,0,0,0,0,0,1,0,1269203261,Invisible Stripes (1939),Crime|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,3166,425464,311078,162516,44004,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1408820482,Failure to Launch (2006),Comedy|Romance
3213,3165,425463,823186,162516,5952,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1175594311,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
3214,3164,425462,477006,162516,160,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1074040573,Congo (1995),Action|Adventure|Mystery|Sci-Fi
3215,3171,425470,129393,162516,2520,3.0,None,None,None,0,...,0,0,0,0,0,1,0,1075736674,Airport (1970),Drama


In [17]:
rankings_hits

,userId,movieId,prediction_x,rank,title_x,genres,Unnamed: 0.1,Unnamed: 0,Column1,rating,...,3&4,1&2&3,2&3&4,1&2&4,1&3&4,1&2&3&4 = 0,1&2&3&4 = 1,timestamp,title_y.1,genres_y.1
0,8619,99402,3.920971,1,Alois Nebel (2011),Animation|Crime|War,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,8619,58998,3.904058,2,Forgetting Sarah Marshall (2008),Comedy|Romance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,8619,32298,3.858332,3,Guess Who (2005),Comedy|Romance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,8619,178153,3.803753,4,Death Trance (2005),Action|Sci-Fi|Thriller,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,8619,135202,3.800667,5,Prime Suspect: Scent of Darkness (1995),(no genres listed),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,162516,106458,3.535159,6,"Tiger from Tjampa, The (Harimau Tjampa) (1953)",Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,162516,178483,3.529783,7,The Marchers (2013),Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,162516,97768,3.526452,8,Free Men (Les hommes libres) (2011),Drama|War,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,162516,7004,3.520524,9,Kindergarten Cop (1990),Action|Comedy|Crime|Thriller,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
from metric.metric import transform_rankings_hits, get_user_dcg, get_dcg

rankings_hits_transformed = transform_rankings_hits(rankings_hits, genres)
rankings_hits_transformed_ideal = transform_rankings_hits(user_ideal_ranks_df, genres)

# user_id = 12
# k = 5
# user_dcg = get_user_dcg(user_id, rankings_hits_transformed, ratings_df, likelihood_dict, k)
# user_idcg = get_user_dcg(user_id, rankings_hits_transformed_ideal, ratings_df, likelihood_dict, k)
# user_idcg

row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5
row.rating is4.5


C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\5 - Ensemble Learning Model\Accuracy Metrics\Group Validation\metric\metric.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rankings_hits_reform.loc[i, genre] = float(row.rating)
C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\5 - Ensemble Learning Model\Accuracy Metrics\Group Validation\metric\metric.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rankings_hits_reform.loc[i, genre] = float(row.rating)
C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\5 - Ensemble Learning Model\A

In [19]:
# alpha-beta-nDCG for the whole test data
dcg = get_dcg(rankings_hits_transformed, ratings_df, likelihood_dict, discarded_users, k)
idcg = get_dcg(rankings_hits_transformed_ideal, ratings_df, likelihood_dict, discarded_users, k)

alpha_beta_ndcg = dcg[0]/idcg[0]
alpha_beta_ndcg

0.010037638283737747

### Group validation

In [20]:
dcg_df = dcg[1]
idcg_df = idcg[1]
all_clusters_list = grouped_clusters.users_per_cluster.to_list()
group_metric = {}

for cluster_id, cluster in enumerate(all_clusters_list):
    # dcg/idcg for the cluster
    cluster_dcg_df = dcg_df.loc[dcg_df['userId'].isin(cluster)]
    cluster_idcg_df = idcg_df.loc[idcg_df['userId'].isin(cluster)]
    cluster_dcg = sum(cluster_dcg_df['DCG'].to_list())
    cluster_idcg = sum(cluster_idcg_df['DCG'].to_list())
    # cluster ab-nDCG
    if cluster_idcg == 0:
        if cluster_dcg == 0:
            cluster_alpha_beta_ndcg = 0
        else:
            cluster_alpha_beta_ndcg = cluster_dcg
    else:
        cluster_alpha_beta_ndcg = cluster_dcg/cluster_idcg
    

    # dcg/idcg for the equivalent (cluster - dataset = rest of the examples)
    cluster_equiv_dcg_df = dcg_df.loc[~dcg_df['userId'].isin(cluster)]
    cluster_equiv_idcg_df = idcg_df.loc[~idcg_df['userId'].isin(cluster)]
    cluster_equiv_dcg = sum(cluster_equiv_dcg_df['DCG'].to_list())
    cluster_equiv_idcg = sum(cluster_equiv_idcg_df['DCG'].to_list())
    # equivalent set ab-nDCG
    if cluster_equiv_idcg == 0:
        if cluster_equiv_dcg == 0:
            cluster_equiv_alpha_beta_ndcg = 0
        else:
            cluster_equiv_alpha_beta_ndcg = cluster_equiv_dcg
    else:
        cluster_equiv_alpha_beta_ndcg = cluster_equiv_dcg/cluster_equiv_idcg

    group_metric[cluster_id] = [cluster_alpha_beta_ndcg, cluster_equiv_alpha_beta_ndcg]

In [21]:
group_metric_df = pd.DataFrame.from_dict(group_metric, orient='index')\
    .reset_index()\
    .rename({'index':'cluster', 0:'cluster-ab-nDCG', 1: 'equiv-ab-nDCG'}, axis=1)
group_metric_df['ab-ndcg'] = alpha_beta_ndcg
group_metric_df['perc-change'] = group_metric_df.apply(lambda x: ((x['cluster-ab-nDCG'] - x['ab-ndcg']) / x['ab-ndcg'])*100, axis=1)
group_metric_df['condition-1'] = group_metric_df.apply(lambda x: (x['cluster-ab-nDCG'] - x['equiv-ab-nDCG']), axis=1)
group_metric_df.to_csv('output/' + dataset_name + '/group_ab1_ndcg_'+dataset_name+'.csv', index=False)